In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
from copy import deepcopy 




from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef as MCC_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score

from catboost import Pool, cv
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

from collections import Counter 

from tqdm import tqdm

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import phik

# Чтение данных

In [3]:
data = pd.read_csv('train.csv', low_memory=False)
data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


# Проверка данных на типы и их изменение

In [219]:
col_info = pd.DataFrame(data.nunique()).rename(columns = {0: 'n_values'})
uniq_values = []
types = []
count_values = []
target_phik = []

for column in tqdm(data.columns):
    
    x, y = data[['target', column]].T.values

    
    non_none_values = data[column].dropna()
    data_type = non_none_values.dtype
    uniq_values.append(data[column].unique())
    count_values.append(len(non_none_values))
    types.append(data_type)
    
    if data_type == 'object':
        data[column] = data[column].fillna(-404).apply(str)
    
    #target_phik.append(phik.phik_from_array(x, y, num_vars=['x']))

col_info['uniq_values'] = uniq_values
col_info['data_type'] = types
col_info['values_count'] = count_values
#col_info['target_phik'] = target_phik

100%|██████████████████████████████████████████████████████████████████████████████| 2666/2666 [01:05<00:00, 40.52it/s]


In [222]:
#col_info.sort_values('target_phik').to_excel('vdv.xlsx')

In [223]:
obj_cols = list(col_info[col_info['data_type'] == 'object'].index)
obj_cols

['report_date',
 'col49',
 'col50',
 'col51',
 'col52',
 'col53',
 'col54',
 'col55',
 'col56',
 'col57',
 'col58',
 'col59',
 'col60',
 'col61',
 'col62',
 'col63',
 'col64',
 'col85',
 'col86',
 'col87',
 'col88',
 'col89',
 'col90',
 'col91',
 'col92',
 'col93',
 'col94',
 'col95',
 'col96',
 'col97',
 'col98',
 'col99',
 'col100',
 'col129',
 'col130',
 'col131',
 'col132',
 'col133',
 'col134',
 'col135',
 'col136',
 'col137',
 'col138',
 'col139',
 'col140',
 'col141',
 'col142',
 'col143',
 'col144',
 'col145',
 'col146',
 'col147',
 'col148',
 'col149',
 'col150',
 'col151',
 'col152',
 'col153',
 'col154',
 'col155',
 'col156',
 'col157',
 'col158',
 'col159',
 'col160',
 'col161',
 'col162',
 'col163',
 'col164',
 'col165',
 'col166',
 'col167',
 'col168',
 'col169',
 'col170',
 'col171',
 'col172',
 'col173',
 'col174',
 'col175',
 'col176',
 'col181',
 'col182',
 'col183',
 'col184',
 'col185',
 'col186',
 'col187',
 'col188',
 'col189',
 'col190',
 'col191',
 'col192',
 'c

# Создание первых фич

## Создание шумовых фич

In [29]:
data['noise_5'] = np.random.randint(low=0, high=4, size=len(data))
data['noise_05'] = np.random.normal(0.5, 0.2)

## Создание лишнего таргета

In [30]:
sorted(Counter(data[['client_id','target']].groupby('client_id').count()['target']).items())

[(1, 1187),
 (2, 1077),
 (3, 935),
 (4, 667),
 (5, 461),
 (6, 267),
 (7, 115),
 (8, 66),
 (9, 25),
 (10, 11),
 (11, 5),
 (12, 1)]

In [31]:
sorted(Counter(data[['client_id','target']].groupby('client_id').sum()['target']).items())

[(0, 4634), (1, 41), (2, 31), (3, 111)]

In [36]:
dict_2 = data[['client_id','target']].groupby('client_id').max().reset_index().set_index('client_id').to_dict()['target']
dict_2

{1: 0,
 2: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 10: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 1,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 87: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 96: 0,
 99: 0,
 101: 0,
 102: 0,
 103: 0,
 105: 0,
 106: 0,
 107: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 118: 0,
 119: 0,
 121: 0,
 122: 0,
 123: 0,
 126: 0,
 128: 0,
 130: 0,
 131: 0,
 133: 0,
 134: 0,
 135: 0,
 136: 0,
 137: 0,
 138: 0,
 139: 0,
 140: 0,
 141: 0,
 143: 0,
 147: 0,
 149: 0,
 151: 0,
 152: 0,
 153: 0,
 154: 0,
 155: 0,
 156: 

In [38]:
np.cos(3.14)

-0.9999987317275395

In [39]:
'2022-11-01'[5:-3]

'11'

In [40]:
data['target_2'] = list(map(lambda x: dict_2[x], data['client_id']))
data['mnth'] = data['report_date'].apply(lambda x: np.cos(int(x[5:-3])/2))
data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2658,col2659,col2660,col2661,col2662,col2663,noise_5,noise_05,target_2,mnth
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.256261,0,0.882691,0,0.708670
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261,0,0.882691,0,0.708670
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.256261,2,0.882691,0,-0.801144
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.258682,3,0.882691,0,-0.210796
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.254164,3,0.882691,0,-0.653644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.276902,2,0.882691,0,-0.936457
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.260516,2,0.882691,1,-0.210796
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.260005,0,0.882691,1,0.540302
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.256261,1,0.882691,0,-0.653644


# Первый слой 🧅

In [ ]:
obj_cols.remove('col1454')
obj_cols.remove('report_date')


In [97]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

our_models = []

need_columns = ['target','client_id','col1454','report_date','target_2'] # столбцы нужные для удаления 

# Определите целевую переменную и группировочную переменную (client_id)
X = data.drop(columns=need_columns).fillna(-404)
y = data['target']
groups = data['client_id']

# Создайте объект StratifiedGroupKFold и задайте количество фолдов
n_splits = 5  # Задайте нужное количество фолдов
sgkf = StratifiedGroupKFold(n_splits=n_splits)

# Проходите по каждому фолду
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y, groups)):
    # Далее вы можете использовать train_idx и val_idx для разделения данных на обучающий и валидационный фолд
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    
    
    
    
    
    X_train = train_data.drop(need_columns, axis=1).fillna(-404)
    y_train = train_data['target']
    
    X_test = val_data.drop(need_columns, axis=1).fillna(-404)
    y_test = val_data['target']
    
    train_pool = Pool(data=X_train, label=y_train,  cat_features = obj_cols)
    test_pool = Pool(data=X_test, label=y_test,  cat_features = obj_cols)

    # Создайте объект модели CatBoostClassifier
    model = CatBoostClassifier(iterations= 100,
          depth= 7,
          loss_function='Logloss', auto_class_weights="Balanced",
        boosting_type='Ordered', l2_leaf_reg= 3,
          eval_metric='AUC',
          learning_rate= 0.05,
            custom_metric = ['MCC','Accuracy','Precision','Recall','F1','NormalizedGini']
          ) 

    # Обучите модель на обучающем Pool
    model.fit(train_pool, eval_set=test_pool,verbose= False,
        plot=True,
         use_best_model= True
            )
    

    our_models += [model]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [98]:

importance_df.Importances.mean()

0.03752345215759853

In [99]:
importance_df = our_models[0].get_feature_importance(prettified=True)
stop_index = importance_df[(importance_df['Importances'] <= 0.00005)].index[0]
set_of_good_features = set(importance_df['Feature Id'][:stop_index].to_list())
for i in range(5): 
    importance_df = our_models[i].get_feature_importance(prettified=True)
    stop_index = importance_df[(importance_df['Importances'] <= 0.00005)].index[0]
    new_set = set(importance_df['Feature Id'][:stop_index].to_list())
    set_of_good_features = set_of_good_features & new_set

In [100]:
list(set_of_good_features)


['col2366',
 'col2388',
 'col2340',
 'col2382',
 'col2360',
 'col2220',
 'col2663',
 'col2294']

# Второй слой 🧅🧅

In [4]:
set_of_good_features = set(['col2366',
 'col2388',
 'col2340',
 'col2382',
 'col2360',
 'col2220',
 'col2663',
 'col2294'])

In [5]:
set_of_good_features

{'col2220',
 'col2294',
 'col2340',
 'col2360',
 'col2366',
 'col2382',
 'col2388',
 'col2663'}

In [6]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

our_models_2 = []

need_columns = ['target','client_id','col1454','report_date'] # столбцы нужные для удаления 

# Определите целевую переменную и группировочную переменную (client_id)
X = data.drop(columns=need_columns).fillna(-404)
y = data['target']
groups = data['client_id']

# Создайте объект StratifiedGroupKFold и задайте количество фолдов
n_splits = 5  # Задайте нужное количество фолдов
sgkf = StratifiedGroupKFold(n_splits=n_splits)

# Проходите по каждому фолду
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y, groups)):
    # Далее вы можете использовать train_idx и val_idx для разделения данных на обучающий и валидационный фолд
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    
    
    
    
    
    X_train = train_data[list(set_of_good_features)]
    y_train = train_data['target']
    
    X_test = val_data[list(set_of_good_features)]
    y_test = val_data['target']
    
    train_pool = Pool(data=X_train, label=y_train)
    test_pool = Pool(data=X_test, label=y_test)

    # Создайте объект модели CatBoostClassifier
    model = CatBoostClassifier(iterations= 100,
          depth= 6,
          loss_function='Logloss', auto_class_weights="Balanced",
        boosting_type='Ordered', l2_leaf_reg= 3,
          eval_metric='AUC',
          learning_rate= 0.01,
            custom_metric = ['MCC','Accuracy','Precision','Recall','F1','NormalizedGini']
          ) 

    # Обучите модель на обучающем Pool
    model.fit(train_pool, eval_set=test_pool,verbose= False,
        plot=True,
         use_best_model= True
            )
    

    our_models_2 += [model]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Feature инжиниринг

In [115]:
good_features = list(set_of_good_features)

def pw_prod(df):
    for col1 in list(set_of_good_features):
        for col2 in list(set_of_good_features):
            
            df[f'{col1}+{col2}'] = list(np.array(df[col1]) + np.array(df[col2]))
            df[f'{col1}-{col2}'] = list(np.array(df[col1]) - np.array(df[col2]))
            df[f'{col1}*{col2}'] = list(np.array(df[col1]) * np.array(df[col2]))
            df[f'{col1}/{col2}'] = list(np.array(df[col1]) / np.array(df[col2]))
            
for col1 in list(set_of_good_features):
    for col2 in list(set_of_good_features):
        good_features += [f'{col1}+{col2}']
        good_features += [f'{col1}-{col2}']
        good_features += [f'{col1}*{col2}']
        good_features += [f'{col1}/{col2}']
            
            
            
good_features = list(set(good_features))
pw_prod(data)

C:\Users\MgDuck\AppData\Local\Temp\ipykernel_28368\2739763497.py:10: RuntimeWarning: divide by zero encountered in true_divide
  df[f'{col1}/{col2}'] = list(np.array(df[col1]) / np.array(df[col2]))
C:\Users\MgDuck\AppData\Local\Temp\ipykernel_28368\2739763497.py:10: RuntimeWarning: invalid value encountered in true_divide
  df[f'{col1}/{col2}'] = list(np.array(df[col1]) / np.array(df[col2]))


In [116]:
data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2294*col2220,col2294/col2220,col2294+col2663,col2294-col2663,col2294*col2663,col2294/col2663,col2294+col2294,col2294-col2294,col2294*col2294,col2294/col2294
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.984070e+07,2.253590e-06,15.256261,14.743739,3.843921,58.533986,30.0,0.0,225.00,1.0
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.867796e+07,4.271573e-07,3.756261,3.243739,0.896915,13.657930,7.0,0.0,12.25,1.0
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.064368e+08,2.405184e-06,16.256261,15.743739,4.100182,62.436251,32.0,0.0,256.00,1.0
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.000000e+04,8.000000e-04,8.258682,7.741318,2.069454,30.926030,16.0,0.0,64.00,1.0
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.232866e+08,2.579644e-06,24.254164,23.745836,6.099943,94.427110,48.0,0.0,576.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.262918e+07,7.661853e-07,5.276902,4.723098,1.384510,18.056928,10.0,0.0,25.00,1.0
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.590430e+08,1.115645e-06,17.260516,16.739484,4.428779,65.255005,34.0,0.0,289.00,1.0
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.072510e+08,9.323925e-07,10.260005,9.739995,2.600054,38.460737,20.0,0.0,100.00,1.0
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.645432e+07,9.875374e-07,6.256261,5.743739,1.537568,23.413594,12.0,0.0,36.00,1.0


# Третий слой 🧅

In [121]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

our_models_3 = []

need_columns = ['target','client_id','col1454','report_date','target_2'] # столбцы нужные для удаления 

# Определите целевую переменную и группировочную переменную (client_id)
X = data.drop(columns=need_columns).fillna(-404)
y = data['target']
groups = data['client_id']

# Создайте объект StratifiedGroupKFold и задайте количество фолдов
n_splits = 5  # Задайте нужное количество фолдов
sgkf = StratifiedGroupKFold(n_splits=n_splits)

# Проходите по каждому фолду
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y, groups)):
    # Далее вы можете использовать train_idx и val_idx для разделения данных на обучающий и валидационный фолд
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    
    
    
    
    
    X_train = train_data[good_features]
    y_train = train_data['target']
    
    X_test = val_data[good_features]
    y_test = val_data['target']
    
    train_pool = Pool(data=X_train, label=y_train)
    test_pool = Pool(data=X_test, label=y_test)

    # Создайте объект модели CatBoostClassifier
    model = CatBoostClassifier(iterations= 200,
          depth= 7,
          loss_function='Logloss', auto_class_weights="Balanced",
        boosting_type='Ordered', l2_leaf_reg= 3,
          eval_metric='AUC',
          learning_rate= 0.05,
            custom_metric = ['MCC','Accuracy','Precision','Recall','F1','NormalizedGini']
          ) 

    # Обучите модель на обучающем Pool
    model.fit(train_pool, eval_set=test_pool,verbose= False,
        plot=True,
         use_best_model= True
            )
    

    our_models_3 += [model]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Эвристика 42

In [124]:
sum_dict = data[['client_id','target']].groupby('client_id').sum().reset_index().set_index('client_id').to_dict()['target']
count_dict = data[['client_id','target']].groupby('client_id').count().reset_index().set_index('client_id').to_dict()['target']

In [169]:
temp_dict = {}
target_3 = []
data = data.sort_values('report_date')
for clid in data.client_id:
    if clid not in temp_dict.keys():
        temp_dict[clid] = 1
    else:
        temp_dict[clid] += 1
    
    if sum_dict[clid] == 0:
        target_3 += [0]
    elif temp_dict[clid] == (count_dict[clid] - sum_dict[clid]):
        target_3 += [1]
    elif temp_dict[clid] < (count_dict[clid] - sum_dict[clid]):
        target_3 += [0]
    else:
        target_3 += [2]
    

In [172]:
Counter(target_3)

Counter({0: 13947, 2: 436, 1: 73})

In [173]:
data['target_3'] = target_3
data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2294/col2220,col2294+col2663,col2294-col2663,col2294*col2663,col2294/col2663,col2294+col2294,col2294-col2294,col2294*col2294,col2294/col2294,target_3
9374,2021-02-01,491,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.191597e-06,10.256261,9.743739,2.562614,39.022657,20.0,0.0,100.0,1.0,0
2289,2021-02-01,359,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.344574e-06,8.256261,7.743739,2.050091,31.218126,16.0,0.0,64.0,1.0,0
8035,2021-02-01,637,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.964839e-06,32.256261,31.743739,8.200364,124.872502,64.0,0.0,1024.0,1.0,0
2286,2021-02-01,2895,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.291846e-06,10.256261,9.743739,2.562614,39.022657,20.0,0.0,100.0,1.0,0
2758,2021-02-01,3302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,2023-02-01,4174,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.869632e-06,14.261248,13.738752,3.657468,53.588980,28.0,0.0,196.0,1.0,0
2901,2023-02-01,3429,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.296181e-06,6.256261,5.743739,1.537568,23.413594,12.0,0.0,36.0,1.0,0
13351,2023-02-01,4000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.523230e-06,16.257217,15.742783,4.115479,62.204184,32.0,0.0,256.0,1.0,0
7283,2023-02-01,5716,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.287614e-07,1.256437,0.743563,0.256437,3.899589,2.0,0.0,1.0,1.0,0


In [177]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold


our_models_42 = []

need_columns = ['target','client_id','col1454','report_date','target_2', 'target_3'] # столбцы нужные для удаления 

# Определите целевую переменную и группировочную переменную (client_id)
X = data.drop(columns=need_columns).fillna(-404)
y = data['target_3']
groups = data['client_id']

# Создайте объект StratifiedGroupKFold и задайте количество фолдов
n_splits = 5  # Задайте нужное количество фолдов
sgkf = StratifiedGroupKFold(n_splits=n_splits)

# Проходите по каждому фолду
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y, groups)):
    # Далее вы можете использовать train_idx и val_idx для разделения данных на обучающий и валидационный фолд
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    
    
    
    
    
    X_train = train_data.drop(need_columns, axis=1).fillna(-404)
    y_train = train_data['target_3']
    
    X_test = val_data.drop(need_columns, axis=1).fillna(-404)
    y_test = val_data['target_3']
    
    train_pool = Pool(data=X_train, label=y_train,  cat_features = obj_cols)
    test_pool = Pool(data=X_test, label=y_test,  cat_features = obj_cols)
    
    

    
    
    # Создайте объект модели CatBoostClassifier
    model = CatBoostClassifier(iterations= 70,
          depth= 6,
          loss_function='MultiClass', auto_class_weights="Balanced",
        boosting_type='Ordered', l2_leaf_reg= 3,
          eval_metric='AUC',
          learning_rate= 0.07,
            custom_metric = ['MCC','Accuracy','Precision','Recall','F1','NormalizedGini']
          ) 

    # Обучите модель на обучающем Pool
    model.fit(train_pool, eval_set=test_pool,verbose= False,
        plot=True,
         use_best_model= True
            )
    

    our_models_42 += [model]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [204]:
roc_auc_score(data['target'],list(map(lambda x:x[1] * 0.5 + x[2] ,our_models_42[0].predict(X,prediction_type='Probability'))))

0.8151065319530423

# Процесс предсказания

In [7]:
tst = pd.read_csv('test.csv', low_memory=False, sep=';')
tst

,report_date,client_id,col1,col2,col3,col4,col5,col6,col7,col8,...,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663,id
0,2023-04-01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4941670.0,4941670.0,NaN,NaN,NaN,4941670.0,4941670.0,4941670.0,0.256919,0
1,2023-05-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11789500.0,11789500.0,NaN,NaN,NaN,11789500.0,11789500.0,11789500.0,0.267221,1
2,2023-05-01,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7000000.0,7000000.0,NaN,NaN,NaN,7000000.0,7000000.0,7000000.0,0.257217,2
3,2023-03-01,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3
4,2023-03-01,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10391000.0,10391000.0,10391000.0,0.257348,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,2023-04-01,4366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3500000.0,3500000.0,3500000.0,0.256261,3637
3638,2023-04-01,8298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3638
3639,2023-03-01,835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256919,3639
3640,2023-03-01,8300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,3640


In [8]:
need_columns = ['id','client_id','col1454','report_date'] # столбцы нужные для удаления 

# Определите целевую переменную и группировочную переменную (client_id)
X = tst[['col2366',
 'col2388',
 'col2340',
 'col2382',
 'col2360',
 'col2220',
 'col2663',
 'col2294']].fillna(-404)
X

,col2366,col2388,col2340,col2382,col2360,col2220,col2663,col2294
0,1.0,30.09,5794341.03,30.09,1.0,6564363.20,0.256919,5.000000
1,2.0,63.51,13981124.40,63.51,2.0,13693099.02,0.267221,3.333333
2,2.0,49.92,11139747.84,49.92,2.0,12417006.76,0.257217,21.000000
3,2.0,37.60,10795110.40,-404.00,-404.0,10785110.30,0.256261,5.000000
4,1.0,37.05,9756932.25,-404.00,-404.0,9649462.80,0.257348,14.000000
...,...,...,...,...,...,...,...,...
3637,1.0,22.73,3906514.18,-404.00,-404.0,4318743.41,0.256261,2.000000
3638,2.0,56.90,16130694.80,-404.00,-404.0,15778680.67,0.256261,7.000000
3639,1.0,30.50,12149858.00,-404.00,-404.0,12619544.28,0.256919,14.000000
3640,2.0,54.63,11431108.98,54.63,2.0,14027694.75,0.257217,3.000000


In [9]:
preds = our_models_2[0].predict(X,prediction_type='Probability')
mmm = np.array(list(map(lambda x: x[1],preds)))
for i in range(1,5):
    preds = our_models_2[i].predict(X,prediction_type='Probability')
    mmm += np.array(list(map(lambda x: x[1],preds)))
    print(np.array(list(map(lambda x: x[1],preds)))[:10])
target = mmm / 5

[0.36464556 0.68242254 0.5716573  0.38038045 0.44812824 0.38258733
 0.51364487 0.51251694 0.28884685 0.56486031]
[0.38058745 0.69456143 0.5998059  0.33900909 0.37256478 0.38372127
 0.54169313 0.46976654 0.28510047 0.55527541]
[0.36342305 0.69574329 0.60173005 0.33603051 0.40775978 0.34769809
 0.48456921 0.48398119 0.26747801 0.52127235]
[0.41471806 0.66224105 0.56277207 0.40391054 0.430351   0.41798006
 0.56776195 0.42119321 0.27394887 0.56345022]


In [10]:
tst['target'] = target
tst

,report_date,client_id,col1,col2,col3,col4,col5,col6,col7,col8,...,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663,id,target
0,2023-04-01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4941670.0,NaN,NaN,NaN,4941670.0,4941670.0,4941670.0,0.256919,0,0.390939
1,2023-05-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11789500.0,NaN,NaN,NaN,11789500.0,11789500.0,11789500.0,0.267221,1,0.666979
2,2023-05-01,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7000000.0,NaN,NaN,NaN,7000000.0,7000000.0,7000000.0,0.257217,2,0.579078
3,2023-03-01,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3,0.374736
4,2023-03-01,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10391000.0,10391000.0,10391000.0,0.257348,4,0.424731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,2023-04-01,4366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3500000.0,3500000.0,3500000.0,0.256261,3637,0.277026
3638,2023-04-01,8298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3638,0.400943
3639,2023-03-01,835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256919,3639,0.409609
3640,2023-03-01,8300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,3640,0.518917


In [11]:
tst[['id','target']].to_csv('sub.csv',sep=';',index=False)

In [ ]:
len(our_models[0].get_feature_importance())

In [ ]:
len(X)

In [276]:
X = X[['col2366',
 'col2388',
 'col2340',
 'col2382',
 'col2360',
 'col2220',
 'col2663',
 'col2294']].fillna(-404)

In [278]:
np.array(list(map(lambda x: x[1],preds)))

array([0.47918002, 0.4850569 , 0.47918002, ..., 0.52819245, 0.47197118,
       0.49681692])